# [Module 4.1] Personalize 캠페인 생성 하기

이 노트북은 이전 노트북에서 생성한 솔류션 버전을 바탕으로 아래와 같은 작업을 합니다.

* 5가지 캠페인 생성
    - user-personalization
    - SIMS
    
--- 
이 노트북의 실행 시간은 약 10분 걸립니다.

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

### Campaign: user-personalization 

In [4]:
create_campaign_response = personalize.create_campaign(
    name = "Movielens-user-personalization-campaign-" + suffix,
    solutionVersionArn = user_personalization_solution_version_arn,
    minProvisionedTPS = 1
)

user_personalization_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:057716757052:campaign/Movielens-user-personalization-campaign-76427",
  "ResponseMetadata": {
    "RequestId": "c0553d6c-cb8f-4eeb-af4b-bbafb77763d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:41:50 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "120",
      "connection": "keep-alive",
      "x-amzn-requestid": "c0553d6c-cb8f-4eeb-af4b-bbafb77763d0"
    },
    "RetryAttempts": 0
  }
}


### Campaign: Similar Items

In [5]:
create_campaign_response = personalize.create_campaign(
    name = "Movielens-similar-items-campaign-" + suffix,
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:ap-northeast-2:057716757052:campaign/Movielens-similar-items-campaign-76427",
  "ResponseMetadata": {
    "RequestId": "210980c9-4883-46ee-ba89-1b934a423cae",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sun, 26 Feb 2023 02:42:13 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "210980c9-4883-46ee-ba89-1b934a423cae"
    },
    "RetryAttempts": 0
  }
}


### 캠페인 생성 및 대기

작동하는 솔루션 버전을 보유하고 있으므로, 이제 애플리케이션과 함께 사용할 캠페인을 작성해야 합니다. 캠페인은 단순히 모델의 호스팅된 사본입니다. 물론 인프라가 프로비저닝되기까지의 시간이 소요됩니다.

In [6]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    
    
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = user_personalization_campaign_arn
    )
    status_user_per = describe_campaign_response["campaign"]["status"]
    print("User_personalization_Campaign: {}".format(status_user_per))

    describe_campaign_response = personalize.describe_campaign(
    campaignArn = sims_campaign_arn
    )
    status_sims = describe_campaign_response["campaign"]["status"]
    print("Sims_Campaign: {}".format(status_sims))
     
    if (status_user_per == "ACTIVE" or status_user_per == "CREATE FAILED")&\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED"):\
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All Campaign creation completed")   

User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE PENDING
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_personalization_Campaign: CREATE IN_PROGRESS
Sims_Campaign: CREATE IN_PROGRESS
-------------------------------------->
User_persona


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [7]:
%store user_personalization_campaign_arn
%store sims_campaign_arn

Stored 'user_personalization_campaign_arn' (str)
Stored 'sims_campaign_arn' (str)
